In [1]:
# Import the Necessary Libs
import pandas as pd
import numpy as np
import getpass
import psycopg2 as ps
import os

In [2]:
# Read the Dataframe 
df = pd.read_csv(r'C:\Users\HP\Downloads\Survey Dataset - Technical Interview.csv')
df.head(5)

,response_id,collection_date,urban_rural,education,income,Vote_Now,Past_Vote,CM_satisfaction,MLA_satisfaction,age,gender,assembly_no,category,weight
0,1,5-24-2017,Rural,12th Pass,"₹ 30,000 - ₹ 50,000",BJP,BJP,Fully Dissatisfied,Fully Dissatisfied,26,Male,211,Gen,0.346969
1,2,5-24-2017,Rural,Graduate,"₹ 20,000 - ₹ 30,000",RLSP,JD(U),Somewhat Satisfied,Fully Dissatisfied,34,Male,189,BC,0.603419
2,3,5-24-2017,Rural,Graduate,"₹ 5,000 - ₹ 10,000",JAP(L),BJP,Fully Dissatisfied,Somewhat Satisfied,25,Male,191,Gen,0.673762
3,4,5-24-2017,Urban,Graduate,"₹ 10,000 - ₹ 20,000",RJD,RJD,Fully Dissatisfied,Somewhat Satisfied,36,Male,194,Gen,1.101298
4,5,5-24-2017,Rural,Graduate,"₹ 50,000 - ₹ 80,000",JD(U),Did not vote,Fully Satisfied,Fully Satisfied,22,Male,176,BC,0.333505


In [3]:
# Clean the column names if any
df.columns = [x.lower().replace(" ","_").replace("?","")\
                 .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","")\
                 .replace(")","").replace(r")","").replace("$","") for x in df.columns]

In [4]:
# Pythom DF Data types are different than Postgres Data types so we need to replace them
replacements = {
    'int64' : 'int',
    'float64' : 'float',
    'datetime64[ns]' : 'varchar',
    'datetime64' : 'datetime',
    'object' : 'varchar'    
}

In [5]:
# Initialize the Connection
hostname = 'localhost'
username = 'postgres'
database = 'python_postgres_github'
pwd = 'Sw@pykoli17'
port_id = 5432

cur = None
conn = None

# In order to hide our password we can use Getpass method
# pwd = getpass.getpass("Enter Password : ")

# In order to handle the Excepetion try catch block is added
try:
    conn = ps.connect(
        host=hostname,
        user=username,
        dbname = database,
        password=pwd,
        port=port_id
    )
     
        
    # Cursor is the main key to execute our all SQL Commands 
    cur = conn.cursor()
    
    # DROP TABLE =>
    drop = '''DROP TABLE IF EXISTS my_table'''
    cur.execute(drop)
    
    # CREATE TABLE =>
    create = "CREATE TABLE my_table (" + ', '.join( " {} {} ".format(n, d) for(n, d) in zip(df.columns, df.dtypes.replace(replacements))) + ");"
    cur.execute(create)
    print('Table Created Successfully !')
    
    # COPY THE DATAFRAME AND UPLOAD IT TO THE DATABASE
    df.to_csv('updated_file.csv', header=df.columns, index=False)
    
    my_file = open('updated_file.csv')
    print('File Opened Successfully')
    
    SQL_STATE = """ COPY my_table FROM STDIN WITH
                    CSV 
                    HEADER
                    DELIMITER AS ','
    """
    cur.copy_expert(sql=SQL_STATE, file=my_file)
    print("File Copied Successfully !!!")
    
    # In order to see the reflection in Postgres we need to Commit the Code
    conn.commit()
    
except Exception as error:
    print(error)

finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

Table Created Successfully !
File Opened Successfully
File Copied Successfully !!!
